In [1]:
# from pytket.extensions.quantinuum import QuantinuumBackend
# # Select the H1-2 emulation device
# machine = 'H1-2E'
# backend_emu = QuantinuumBackend(device_name=machine)
# backend_emu.login()

In [2]:
# General imports
import time
import numpy as np

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# The IBM Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Session

# SciPy minimizer routine
from scipy.optimize import minimize

# Plotting functions
import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'

from qiskit_nature.second_q.hamiltonians.lattices import LineLattice, BoundaryCondition
from qiskit_nature.second_q.hamiltonians.ising_model import IsingModel

from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import SPSA

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator

service = QiskitRuntimeService(channel="ibm_cloud")

# Initial State

In [4]:
psi1 = RealAmplitudes(num_qubits=10, reps=2)

# Hamiltonian

## Hamiltonian 1

In [5]:
line_lattice = LineLattice(num_nodes=10, boundary_condition=BoundaryCondition.OPEN)

ising_model = IsingModel(
    line_lattice.uniform_parameters(
        uniform_interaction=-1.0,
        uniform_onsite_potential=-1.0,
    ),
)

In [6]:
Ham_1 = ising_model.second_q_op().simplify()
print(Ham_1)

Spin Operator
spin=1/2, number spins=10, number terms=19
  (-1+0j) * ( Z_0 Z_1 )
+ (-1+0j) * ( Z_1 Z_2 )
+ (-1+0j) * ( Z_2 Z_3 )
+ (-1+0j) * ( Z_3 Z_4 )
+ (-1+0j) * ( Z_4 Z_5 )
+ (-1+0j) * ( Z_5 Z_6 )
+ (-1+0j) * ( Z_6 Z_7 )
+ (-1+0j) * ( Z_7 Z_8 )
+ (-1+0j) * ( Z_8 Z_9 )
+ (-1+0j) * ( X_0 )
+ (-1+0j) * ( X_1 )
+ (-1+0j) * ( X_2 )
+ (-1+0j) * ( X_3 )
+ (-1+0j) * ( X_4 )
+ (-1+0j) * ( X_5 )
+ (-1+0j) * ( X_6 )
+ (-1+0j) * ( X_7 )
+ (-1+0j) * ( X_8 )
+ (-1+0j) * ( X_9 )


In [7]:
# hamiltonian = SparsePauliOp.from_list(
#     [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XX", 0.1810)]
# )

# Step 1 - Use VQE to find the eigenvalue and eigenvectors of the Hamiltonian

In [8]:
VQE(Ham_1, RealAmplitudes(10), SPSA(1000))

In [15]:
psi1 = RealAmplitudes(num_qubits=10, reps=2)

In [14]:

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    estimator = Estimator(session=session)

    theta1 = [0, 1, 1, 2, 3, 5]*6

    # calculate [ <psi1(theta1)|H1|psi1(theta1)> ]
    psi1_H1 = estimator.run(circuits=[psi1], observables=[Ham_1], parameter_values=[theta1])
    print(psi1_H1.result())

    # calculate [ <psi1(theta1)|H2|psi1(theta1)>, <psi1(theta1)|H3|psi1(theta1)> ]
    psi1_H23 = estimator.run(
        circuits=[psi1, psi1],
        observables=[Ham_1],
        parameter_values=[theta1]*2
    )
    print(psi1_H23.result())
    # Close the session only if all jobs are finished
    # and you don't need to run more in the session
    session.close()

QiskitError: 'Invalid input data for Pauli.'

In [13]:
[0, 1, 1, 2, 3, 5]*5

[0,
 1,
 1,
 2,
 3,
 5,
 0,
 1,
 1,
 2,
 3,
 5,
 0,
 1,
 1,
 2,
 3,
 5,
 0,
 1,
 1,
 2,
 3,
 5,
 0,
 1,
 1,
 2,
 3,
 5]